In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Function to load and preprocess images
def load_and_preprocess_data(directory):
    images = []
    labels = []
    emotion_labels = os.listdir(directory)

    for label, emotion in enumerate(emotion_labels):
        emotion_dir = os.path.join(directory, emotion)
        for img_name in os.listdir(emotion_dir):
            img_path = os.path.join(emotion_dir, img_name)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
            img = cv2.resize(img, (48, 48))  # Resize images to a common size
            images.append(img)  # Do not flatten the image for RNN
            labels.append(label)

    return np.array(images), np.array(labels)

# Load and preprocess the data
root_dir = '/kaggle/input/emotion-detection-fer/'
train_dir = os.path.join(root_dir, 'train')
test_dir = os.path.join(root_dir, 'test')

X_train, y_train = load_and_preprocess_data(train_dir)
X_test, y_test = load_and_preprocess_data(test_dir)

# Normalize pixel values to be between 0 and 1
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# One-hot encode the labels
num_classes = len(os.listdir(train_dir))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Reshape data for LSTM
X_train_lstm = X_train.reshape(X_train.shape[0], 48, 48)
X_test_lstm = X_test.reshape(X_test.shape[0], 48, 48)

# Split the training data into training and validation sets
X_train_lstm, X_val_lstm, y_train_lstm, y_val_lstm = train_test_split(X_train_lstm, y_train, test_size=0.2, random_state=42)

# Define the LSTM model
model_lstm = Sequential()
model_lstm.add(LSTM(256, input_shape=(48, 48), return_sequences=True))
model_lstm.add(Dropout(0.5))
model_lstm.add(LSTM(128))
model_lstm.add(Dropout(0.5))
model_lstm.add(Dense(num_classes, activation='softmax'))

# Compile the LSTM model
model_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the LSTM model
history_lstm = model_lstm.fit(X_train_lstm, y_train_lstm, epochs=100, batch_size=32, validation_data=(X_val_lstm, y_val_lstm))

# Evaluate the LSTM model on the test set
test_loss_lstm, test_accuracy_lstm = model_lstm.evaluate(X_test_lstm, y_test)
print(f'Test Accuracy (LSTM): {test_accuracy_lstm}')

# Visualize LSTM training history
plt.plot(history_lstm.history['accuracy'], label='Training Accuracy (LSTM)')
plt.plot(history_lstm.history['val_accuracy'], label='Validation Accuracy (LSTM)')
plt.title('Training and Validation Accuracy (LSTM)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


Epoch 1/100
718/718 [==============================] - 122s 164ms/step - loss: 1.7742 - accuracy: 0.2770 - val_loss: 1.7256 - val_accuracy: 0.3060
Epoch 2/100
718/718 [==============================] - 117s 163ms/step - loss: 1.7065 - accuracy: 0.3202 - val_loss: 1.6846 - val_accuracy: 0.3426
Epoch 3/100
718/718 [==============================] - 117s 163ms/step - loss: 1.6708 - accuracy: 0.3399 - val_loss: 1.6704 - val_accuracy: 0.3405
Epoch 4/100
718/718 [==============================] - 132s 184ms/step - loss: 1.6290 - accuracy: 0.3630 - val_loss: 1.6005 - val_accuracy: 0.3687
Epoch 5/100
718/718 [==============================] - 138s 192ms/step - loss: 1.5930 - accuracy: 0.3788 - val_loss: 1.5734 - val_accuracy: 0.3819
Epoch 6/100
718/718 [==============================] - 141s 196ms/step - loss: 1.5707 - accuracy: 0.3866 - val_loss: 1.5896 - val_accuracy: 0.3713
Epoch 7/100
718/718 [==============================] - 139s 193ms/step - loss: 1.5522 - accuracy: 0.3956 - val_loss: 1